#Libraries
Reference: https://github.com/shankarpandala/lazypredict

In [ ]:
!pip install lazypredict
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
from lazypredict.Supervised import LazyRegressor
from google.colab import drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
sag

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Open the dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Agrimonia/Parte2/Kfold/AGC_Dataset.csv', nrows= 1000)
#Replace missing data with the mean
#df =  df.fillna(df.mean())
pd.set_option('display.max_columns', None)

df.dropna(inplace=True)
df.head()

,Latitude,Longitude,Time,Altitude,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_mode_wind_direction_10m,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_mode_wind_direction_100m,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use
0,44.35,8.05,2016-01-01,595.27,3.480e+00,6.320e-01,1.016e+00,SE,1.738e-04,0,9.469e+04,5.026e+06,9.100e+01,9.639e+01,9.914e+01,1.40,1.92,SW,458.80,29.28,1.10,0.05,0.02,1.33,2.85,3.11,0.16,1.058e+01,1.435e+01,1.045e+00,1.707e+00,311
1,44.45,8.05,2016-01-01,635.14,3.672e+00,5.946e-01,1.082e+00,SE,1.150e-04,0,9.685e+04,4.924e+06,9.123e+01,9.614e+01,9.865e+01,1.55,2.48,SW,425.60,32.96,1.07,0.01,0.00,1.15,0.81,0.94,0.07,1.029e+01,1.724e+01,1.002e+00,1.693e+00,311
2,44.55,8.05,2016-01-01,596.34,3.333e+00,6.485e-01,1.353e+00,SE,6.924e-05,0,9.705e+04,4.870e+06,9.157e+01,9.618e+01,9.864e+01,1.44,2.72,SW,389.10,23.42,2.50,0.01,0.00,2.58,0.92,1.06,0.08,1.708e+01,1.205e+01,1.126e+00,2.602e+00,243
3,44.65,8.05,2016-01-01,366.72,3.187e+00,6.589e-01,1.470e+00,SE,5.552e-05,0,9.755e+04,4.870e+06,9.168e+01,9.629e+01,9.887e+01,1.45,2.80,SW,383.20,23.17,1.92,0.30,0.13,2.76,1.73,3.26,0.71,1.885e-03,1.178e+01,1.285e+00,2.403e+00,221
4,44.75,8.05,2016-01-01,208.41,3.327e+00,6.591e-01,1.601e+00,SE,5.356e-05,0,9.950e+04,4.842e+06,9.202e+01,9.655e+01,9.906e+01,1.56,3.13,SE,360.40,25.11,3.29,0.46,0.21,4.45,2.66,7.92,0.75,7.368e+01,3.508e+01,1.585e+00,2.097e+00,221


In [ ]:
df = df.apply(lambda x: pd.to_numeric(x, errors='coerce'))


In [ ]:
df.head()

,Latitude,Longitude,Time,Altitude,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_mode_wind_direction_10m,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_mode_wind_direction_100m,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use
0,44.35,8.05,NaN,595.27,3.48,0.63,1.02,NaN,0.00,0,94690.00,5026000.00,91.00,96.39,99.14,1.40,1.92,NaN,458.80,29.28,1.10,0.05,0.02,1.33,2.85,3.11,0.16,10.58,14.35,1.04,1.71,311
1,44.45,8.05,NaN,635.14,3.67,0.59,1.08,NaN,0.00,0,96850.00,4924000.00,91.23,96.14,98.65,1.55,2.48,NaN,425.60,32.96,1.07,0.01,0.00,1.15,0.81,0.94,0.07,10.29,17.24,1.00,1.69,311
2,44.55,8.05,NaN,596.34,3.33,0.65,1.35,NaN,0.00,0,97050.00,4870000.00,91.57,96.18,98.64,1.44,2.72,NaN,389.10,23.42,2.50,0.01,0.00,2.58,0.92,1.06,0.08,17.08,12.05,1.13,2.60,243
3,44.65,8.05,NaN,366.72,3.19,0.66,1.47,NaN,0.00,0,97550.00,4870000.00,91.68,96.29,98.87,1.45,2.80,NaN,383.20,23.17,1.92,0.30,0.13,2.76,1.73,3.26,0.71,0.00,11.78,1.28,2.40,221
4,44.75,8.05,NaN,208.41,3.33,0.66,1.60,NaN,0.00,0,99500.00,4842000.00,92.02,96.55,99.06,1.56,3.13,NaN,360.40,25.11,3.29,0.46,0.21,4.45,2.66,7.92,0.75,73.68,35.08,1.58,2.10,221


# Try all the possible models

##Select data

In [ ]:
X = df[['Latitude', 'Longitude', 'Time', 'Altitude', 'WE_temp_2m', 'WE_wind_speed_10m_mean', 'WE_wind_speed_10m_max', 'WE_mode_wind_direction_10m', 'WE_tot_precipitation', 'WE_precipitation_t', 'WE_surface_pressure', 'WE_solar_radiation', 'WE_rh_min', 'WE_rh_mean', 'WE_rh_max', 'WE_wind_speed_100m_mean', 'WE_wind_speed_100m_max', 'WE_mode_wind_direction_100m', 'WE_blh_layer_max', 'WE_blh_layer_min', 'EM_nh3_agr_soils', 'EM_nh3_agr_waste_burn', 'EM_nh3_sum', 'EM_nox_traffic', 'EM_nox_sum', 'EM_so2_sum', 'LI_pigs', 'LI_bovine', 'LA_hvi', 'LA_lvi', 'LA_land_use']]
y = df[['EM_nh3_livestock_mm']]


## Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)


#Regression

In [ ]:
offset = int(X.shape[0] * 0.9)

X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)


 93%|█████████▎| 39/42 [00:43<00:03,  1.29s/it]

[19:54:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


100%|██████████| 42/42 [00:43<00:00,  1.04s/it]


In [ ]:
print(models)

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
GradientBoostingRegressor                    0.99       0.99  0.27        0.84
XGBRegressor                                 0.99       0.99  0.31        0.24
HuberRegressor                               0.98       0.99  0.34        0.10
LinearRegression                             0.98       0.99  0.34        0.04
TransformedTargetRegressor                   0.98       0.99  0.34        0.02
LassoLarsCV                                  0.98       0.99  0.34        0.12
BayesianRidge                                0.98       0.99  0.34        0.02
Ridge                                        0.98       0.99  0.35        0.02
RidgeCV                                      0.98       0.99  0.35        0.03
LinearSVR                                    0.98       0.99  0.35        0.14
LassoLarsIC                                  0.98   